\### Project status <br>
Obs: Notebook criado para o projeto job-seeker, será utilizado até as funcionalidade abaixo estarem em produção

- [ ] Transpor dataframe para o Notion
- [ ] Fazer todas as chamadas aos sites de recrutamento via API

In [1]:
# Install libs and clear output
from IPython.display import clear_output
%pip install pytz datetime requests openpyxl pandas bs4
clear_output(wait=False)

In [2]:
# Remove pandas FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [28]:
# Import libs
import os
import re
import json
import pytz
import datetime
import requests
import openpyxl
import pandas as pd
from bs4 import BeautifulSoup

# Import my libs
import modules.filters as filters
import modules.handler as handler
import modules.notion as notion

In [4]:
# Define function to get html from website through webscraping
def website_request(url, site_type):
    if(site_type == 'Workable'):
        #POST request
        page = requests.post(url)
        data = page.json()
    else:
        #GET request
        page = requests.get(url)
        data = BeautifulSoup(page.content, "html.parser")
    return data

In [5]:
# Import companies dataframe and adjust some URLs for requests
company_df = pd.read_csv('./companies.csv')
company_df.loc[company_df['Site'].str[-1] == '/', 'Site'] = company_df['Site'].str[:-1]
company_df.loc[company_df['Tipo de Site'] == 'Kenoby', 'Site'] = company_df['Site'] + '/position'
company_df.loc[company_df['Tipo de Site'] == 'Workable', 'Site'] = company_df['Site'].apply(lambda x: x.split('/')[-1])
company_df.loc[company_df['Tipo de Site'] == 'Workable', 'Site'] = 'https://apply.workable.com/api/v3/accounts/' + company_df['Site'] + '/jobs'

In [6]:
# Define function to get jobs
def get_array_of_jobs(response, company_name, company_type, site_type, site_url):
    results = []
    if (site_type == 'Gupy'): results = handler.treat_gupy(response, company_name, company_type, site_type, site_url)
    if (site_type == 'Lever'): results = handler.treat_lever(response, company_name, company_type, site_type, site_url)
    if (site_type == 'Greenhouse'): results = handler.treat_greenhouse(response, company_name, company_type, site_type, site_url)
    if (site_type == 'Kenoby'): results = handler.treat_kenoby(response, company_name, company_type, site_type, site_url)
    if (site_type == 'Workable'): results = handler.treat_workable(response, company_name, company_type, site_type, site_url)
    return pd.DataFrame(results)  

In [7]:
# Start collecting jobs
full_job_df = pd.DataFrame()
for n in company_df.index:
    website_response = website_request(company_df['Site'][n], company_df['Tipo de Site'][n])
    jobs = get_array_of_jobs(website_response, company_df['Empresa'][n], company_df['Tipo de Empresa'][n], company_df['Tipo de Site'][n], company_df['Site'][n])
    full_job_df = pd.concat([full_job_df, jobs], ignore_index=True)

In [22]:
# Remove trailing spaces
full_job_df['title'] = full_job_df['title'].str.replace('  ', ' ')
full_job_df['title'] = full_job_df['title'].str.strip()

In [23]:
bkp_full_job_df = full_job_df

In [34]:
full_job_df = bkp_full_job_df

In [35]:
# Define function to filter dict
def filter_dict(df, categories, column_name):
    for key in categories.keys():
        fixed_filter1 = (' ' + ' | '.join([filter for filter in categories[key]]) + ' ').upper()
        fixed_filter2 = (' ' + '| '.join([filter for filter in categories[key]])).upper()
        fixed_filter3 = (' |'.join([filter for filter in categories[key]]) + ' ').upper()
        df.loc[df['title'].str.upper().str.contains(fixed_filter1), column_name] = key
        df.loc[df['title'].str.upper().str.contains(fixed_filter2), column_name] = key
        df.loc[df['title'].str.upper().str.contains(fixed_filter3), column_name] = key

In [36]:
# Apply filter dict    
filter_dict(full_job_df, filters.category_dict, 'category')
filter_dict(full_job_df, filters.category_level, 'level')

In [37]:
# Get now date/time and export dataframe to excel file
now_sao_paulo = pytz.timezone('America/Sao_Paulo').localize(datetime.datetime.now())
now_for_filename = now_sao_paulo.strftime("%Y_%m_%d-%H%M%S")
full_job_df.to_excel(rf'./results/xlsx/jobs_{now_for_filename}.xlsx', encoding='utf-8', index=False)
#full_job_df.to_json('src/results/json/jobs.json', force_ascii=False, orient = 'records')